In [8]:
api_key = "ml97jh99o23eay57z38r0gba871dc13wr3csqg39bw87ywd7d35qd38lfjca"

In [9]:
import requests
import json
from datetime import datetime

# 1. Load Existing Data from the provided file
with open('rounded_structured_metal_data.json', 'r') as file:
    data = json.load(file)
    metals_data = data["Metals"]

# Connect to API and define API parameters
url_1 = f"https://metals-api.com/api/latest?access_key={api_key}&base=AUD&symbols=ANTIMONY%2CLCO%2CLEAD%2CLITHIUM%2CMO"
url_2 = f"https://metals-api.com/api/latest?access_key={api_key}&base=AUD&symbols=NI%2CTIN%2CTUNGSTEN%2CXAG%2CXAU"
url_3 = f"https://metals-api.com/api/latest?access_key={api_key}&base=AUD&symbols=XCU%2CXPD%2CXPT%2CZNC"

response_1 = requests.request("GET", url_1)
response_2 = requests.request("GET", url_2)
response_3 = requests.request("GET", url_3)

# Combine data from API responses
data = response_1.json()
data['rates'].update(response_2.json()['rates'])
data['rates'].update(response_3.json()['rates'])

# Adjust rates
for key, value in data['rates'].items():
    if key not in ['XAU', 'XAG', 'XPD', 'XPT', 'XCU']:
        data['rates'][key] = 1 / value

# Conversion calculations
data['rates']['TUNGSTEN'] *= 3215000
for key in ['ANTIMONY', 'LEAD', 'NI', 'XCU', 'TIN']:
    data['rates'][key] *= 35270
for key in ['LCO','LITHIUM','MO','ZNC']:
    data['rates'][key] *= 32150

# Update metals_data with the new prices
now = datetime.now().strftime('%Y-%m-%d')
for metal in metals_data:
    metal_symbol = metal["Metal"]
    if metal_symbol in data["rates"]:
        # Round the price to 2 decimal places
        rounded_price = round(data["rates"][metal_symbol], 2)
        metal["Prices"].append({
            "Date_Collected": now,
            "Price": rounded_price
        })

# Save the updated data back to the file
with open('rounded_structured_metal_data.json', 'w') as file:
    json.dump({"Metals": metals_data}, file)

In [13]:
import json
from IPython.display import display, HTML
from datetime import datetime, timedelta

# Load the data from the file
with open('rounded_structured_metal_data.json', 'r') as file:
    data = json.load(file)
    metals_data = data["Metals"]

# Define periods for price changes
periods = {
    "1 Day": 1,
    "1 Week": 7,
    "1 Month": 30,
    "3 Months": 90,
    "1 Year": 365
}

# Process data to get percentage and numerical changes
dashboard_data = []
for metal in metals_data:
    metal_name = metal["Metal_Name"]
    unit = metal["Unit"]
    prices = metal["Prices"]
    
    # Sort prices by date
    prices = sorted(prices, key=lambda x: datetime.strptime(x["Date_Collected"], '%Y-%m-%d'), reverse=True)
    
    current_price = prices[0]["Price"]
    price_changes = {}
    current_date = datetime.strptime(prices[0]["Date_Collected"], '%Y-%m-%d')
    
    for label, days in periods.items():
        target_date = current_date - timedelta(days=days)
        
        # Find price closest to target date
        price_on_target_date = next((p["Price"] for p in prices if datetime.strptime(p["Date_Collected"], '%Y-%m-%d') <= target_date), None)
        
        if price_on_target_date:
            change_value = current_price - price_on_target_date
            percent_change = (change_value / price_on_target_date) * 100
            price_changes[label] = {
                "Numerical": round(change_value, 2),
                "Percentage": f"{round(percent_change, 2)}%"
            }
        else:
            price_changes[label] = {
                "Numerical": "-",
                "Percentage": "-"
            }
    
    dashboard_data.append({
        "Metal Name": metal_name,
        "Unit": unit,
        "Current Price": current_price,
        "Price Changes": price_changes
    })

# Convert dashboard data to an HTML table with conditional formatting
def create_dashboard_table(data):
    html = ["<table width=100%>"]
    html.append("<tr>")
    headers = ["Metal Name", "Unit", "Current Price"]
    for period in ["1 Day", "1 Week", "1 Month", "3 Months", "1 Year"]:
        headers.extend([f"{period} Change (Num)", f"{period} Change (%)"])
    html.append("".join([f"<th>{header}</th>" for header in headers]))
    html.append("</tr>")
    
    for metal_data in data:
        html.append("<tr>")
        html.append(f"<td>{metal_data['Metal Name']}</td>")
        html.append(f"<td>{metal_data['Unit']}</td>")
        html.append(f"<td>${metal_data['Current Price']:,.2f}</td>")  # Format as dollar
        for period_change in metal_data["Price Changes"].values():
            # Determine color based on value
            color = ""
            if period_change["Numerical"] != "-" and period_change["Numerical"] != "N/A":
                if float(period_change["Numerical"]) < 0:
                    color = "red"
                else:
                    color = "green"
            html.append(f"<td style='color:{color}'>${period_change['Numerical'] if period_change['Numerical'] not in ['-', 'N/A'] else period_change['Numerical']}</td>")  # Format as dollar and apply color
            html.append(f"<td style='color:{color}'>{period_change['Percentage']}</td>")  # Apply color
        html.append("</tr>")
    
    html.append("</table>")
    return ''.join(html)

# Create and display the dashboard
dashboard_html = create_dashboard_table(dashboard_data)
display(HTML(dashboard_html))

Metal Name,Unit,Current Price,1 Day Change (Num),1 Day Change (%),1 Week Change (Num),1 Week Change (%),1 Month Change (Num),1 Month Change (%),3 Months Change (Num),3 Months Change (%),1 Year Change (Num),1 Year Change (%)
Antimony,t,"$18,724.52",$58.56,0.31%,$-,-,$-,-,$-,-,$-,-
Cobalt,t,"$54,483.43",$197.04,0.36%,$-,-,$-,-,$-,-,$-,-
Lead,t,"$3,639.71",$13.17,0.36%,$-,-,$-,-,$-,-,$-,-
Lithium,t,"$42,914.28",$155.2,0.36%,$-,-,$-,-,$-,-,$-,-
Molybdenum,t,"$75,773.91",$274.03,0.36%,$-,-,$-,-,$-,-,$-,-
Nickel,t,"$31,256.11",$113.03,0.36%,$-,-,$-,-,$-,-,$-,-
Tin,t,"$42,054.55",$152.09,0.36%,$-,-,$-,-,$-,-,$-,-
Tungsten,t,"$48,064.00",$173.82,0.36%,$-,-,$-,-,$-,-,$-,-
Silver,oz,$34.44,$0.49,1.44%,$-,-,$-,-,$-,-,$-,-
Gold,oz,"$2,924.21",$32.27,1.12%,$-,-,$-,-,$-,-,$-,-
